In [ ]:
# -*- coding: utf-8 -*-
"""
Stationarity Test for BTC/ETH Spread

This notebook:
1. Loads BTCUSDT and ETHUSDT close prices
2. Computes ratio and difference series
3. Plots raw series, rolling means, and residuals
4. Runs stationarity tests (ADF & KPSS)
"""

import pandas as pd
import plotly.graph_objects as go
from statsmodels.tsa.stattools import adfuller, kpss

# ======================================
# Load Data
# ======================================
btc = pd.read_csv(r'C:\Users\amirs\OneDrive\Desktop\myAlgoCode\detector\BTCUSDT_1m_1000d.csv')
eth = pd.read_csv(r'C:\Users\amirs\OneDrive\Desktop\myAlgoCode\detector\ETHUSDT_1m_1000d.csv')

btc_price = btc['close']
eth_price = eth['close']

# ======================================
# Helper: Stationarity Tests
# ======================================
def run_stationarity_tests(series, name="Series", sample_step=10):
    """
    Run KPSS and ADF tests on a given series.
    Subsamples the series for ADF to avoid memory issues.
    """
    series = series.dropna()

    print(f"\n=== {name} ===")

    # KPSS Test
    print("\n--- KPSS Test ---")
    kpss_result = kpss(series, regression='c', nlags="auto")
    print(f"Statistic : {kpss_result[0]:.4f}")
    print(f"p-value   : {kpss_result[1]:.4f}")
    print(f"Lags Used : {kpss_result[2]}")
    print("Critical Values:")
    for key, value in kpss_result[3].items():
        print(f"   {key} : {value:.4f}")
    print("=> Stationary" if kpss_result[1] > 0.05 else "=> Non-stationary")

    # ADF Test
    print("\n--- Augmented Dickey-Fuller Test ---")
    adf_result = adfuller(series[::sample_step])
    print(f"ADF Statistic : {adf_result[0]:.4f}")
    print(f"p-value       : {adf_result[1]:.4f}")
    print(f"Lags Used     : {adf_result[2]}")
    print(f"Observations  : {adf_result[3]}")
    print("Critical Values:")
    for key, value in adf_result[4].items():
        print(f"   {key} : {value:.4f}")
    print("=> Stationary" if adf_result[1] < 0.05 else "=> Non-stationary")


# ======================================
# Ratio Series (BTC/ETH)
# ======================================
z_ratio = btc_price / eth_price
rolling_mean_ratio = z_ratio.rolling(60*24).mean()
ratio_resid = z_ratio - rolling_mean_ratio

# Plot ratio & rolling mean
fig = go.Figure()
fig.add_trace(go.Scatter(y=z_ratio, mode="lines", name="BTC/ETH Ratio", line=dict(color="green")))
fig.add_trace(go.Scatter(y=rolling_mean_ratio, mode="lines", name="Rolling Mean (1d)", line=dict(color="red")))
fig.update_layout(title="BTC/ETH Ratio with Rolling Mean", template="plotly")
fig.show()

# Plot residuals
fig = go.Figure()
fig.add_trace(go.Scatter(y=ratio_resid, mode="lines", name="Residuals", line=dict(color="blue")))
fig.update_layout(title="BTC/ETH Ratio Residuals", template="plotly")
fig.show()

# Run tests
run_stationarity_tests(ratio_resid, name="BTC/ETH Ratio Residuals")

# ======================================
# Difference Series (BTC - ETH)
# ======================================
z_diff = btc_price - eth_price
rolling_mean_diff = z_diff.rolling(60*24).mean()
diff_resid = z_diff - rolling_mean_diff
diff_resid_mean = diff_resid.rolling(60*24*30).mean()  # 30-day rolling mean of residuals

# Plot diff residuals
fig = go.Figure()
fig.add_trace(go.Scatter(y=diff_resid, mode="lines", name="Residuals", line=dict(color="green")))
fig.add_trace(go.Scatter(y=diff_resid_mean, mode="lines", name="Rolling Mean (30d)", line=dict(color="red")))
fig.update_layout(title="BTC-ETH Difference Residuals", template="plotly")
fig.show()

# Run tests
run_stationarity_tests(diff_resid, name="BTC-ETH Difference Residuals")
